In [1]:
#!/usr/bin/env python
# coding: utf-8

import functools
import nest_asyncio
import os
import traceback
import pandas as pd
import telebot
from icecream import ic
from IPython import get_ipython
from IPython.utils.capture import capture_output
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain.chat_models import ChatOpenAI
from typing import TypedDict, Annotated, Sequence
import operator
from langchain import hub
from gsheet_handler import GSheetHandler
import importlib
import pandas as pd

nest_asyncio.apply()
module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')

no docx
no docx


In [2]:
example_dialog = ggl.extract_data_from_google_sheet('https://docs.google.com/spreadsheets/d/1sBAFF62fAjSdPrGYTWgpjmBjLRiFCb8DNAX8ZypI-_U/edit?gid=0#gid=0', 
    0)
example_dialog

,Human,AI
0,Интересует программа «Земля за 1 рубль для про...,Это программа для создания новых производств н...
1,Где найти более подробную информацию о конкрет...,"Ознакомиться с земельными участками, потенциал..."
2,Как подать заявку на данную меру поддержки?,Подать заявку можно через личный кабинет на са...
3,Интересует возможность вступления в реестр аль...,Сервис Реестр альтернативных поставщиков напра...
4,Хочу узнать информацию о предоставлении столич...,Статус промышленного комплекса присваивается ю...
5,Как подать заявку на присвоение статуса промыш...,Статус промышленного комплекса присваивается в...
6,Необходима помощь в поиске площадки в Москве д...,Для подбора помещений с целью локализации ваше...
7,Интересуют льготные займы,Московский Фонд Поддержки промышленности и пре...
8,Интересует программа компенсации по кредитам,Московский Фонд поддержки промышленности и пре...
9,Как зарегистрироваться на Инвестиционном портале?,Для регистрации на Портале необходимо в верхне...


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage

def create_training_prompt(dialogs):
    instruction_message = SystemMessage(content="Ты консультант по инвестиционным программам правительства Москвы.")
    examples_message_content = "Примеры диалогов:\n"
    
    for i, row in dialogs.iterrows():
        examples_message_content += f"Человек: {row['Human']}\nAI: {row['AI']}\n"

    examples_message = SystemMessage(content=examples_message_content)
    
    messages = [instruction_message, examples_message]
    prompt_template = ChatPromptTemplate.from_messages(messages)
    return prompt_template

# Пример использования функции с `example_dialog`
import pandas as pd


training_prompt = create_training_prompt(example_dialog)
formatted_messages = training_prompt.format_messages()

for message in formatted_messages:
    print(f"{message.type}: {message.content}")


system: Ты консультант по инвестиционным программам правительства Москвы.
system: Примеры диалогов:
Человек: Интересует программа «Земля за 1 рубль для производства»
AI: Это программа для создания новых производств на территории города Москвы. По программе «Земля за 1 рубль для производства» арендная плата на весь срок аренды земельного участка, оформляемого для проектирования и строительства/реконструкции объектов промышленно-производственного назначения, устанавливается в размере 1 (одного) рубля в год. 
В случае возникновения дополнительных вопросов готовы переключить Вас на соответствующего специалиста.
Человек: Где найти более подробную информацию о конкретном земельном участке?
AI: Ознакомиться с земельными участками, потенциально доступными для строительства производственных объектов, можно на Инвестиционном портале Москвы. Для этого необходимо на главной странице Инвестиционного портала города Москвы investmoscow.ru (инвестмоскоу точка ру) выбрать пункт 4 «Земля за 1 рубль», да

In [4]:
#1/0


In [ ]:
import json
import os
import telebot
import traceback
from icecream import ic
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# Load config
with open('config.json') as config_file:
    config = json.load(config_file)

os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "nlmk_bot"
TELEGRAMM_API_KEY = config["TELEGRAM_BOT_TOKEN"]

# Create bot
bot = telebot.TeleBot(TELEGRAMM_API_KEY)

# Initialize ChatOpenAI model
chat = ChatOpenAI()

# Create training prompt
def create_training_prompt(dialogs):
    messages = [SystemMessage(content="Ты консультант по мерам поддержки и инвестиционным проектам правительства Москвы.")]
    for i, row in dialogs.iterrows():
        messages.append(HumanMessage(content=row['Human']))
        messages.append(AIMessage(content=row['AI']))
    return messages

training_messages = create_training_prompt(example_dialog)

# History of messages
message_history = training_messages.copy()

# Function to create a prompt with history
def create_prompt_with_history(history, user_input):
    messages = history + [HumanMessage(content=user_input)]
    return ChatPromptTemplate.from_messages(messages)

# Function to handle messages
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    ic(f"Received message: {message.text}")

    # Check if the message is directed to the bot
    if message.reply_to_message and message.reply_to_message.from_user.id == bot.get_me().id:
        bot_called = True
    elif f"@{bot.get_me().username}" in message.text:
        bot_called = True
    else:
        bot_called = False

    if bot_called:
        try:
            # Create prompt with history and user input
            prompt_with_history = create_prompt_with_history(message_history, message.text)
            formatted_messages = prompt_with_history.format_messages(user_input=message.text)
            response = chat.invoke(formatted_messages)
            ic(f"Generated response: {response.content}")
            
            # Add the new user message and AI response to the history
            message_history.append(HumanMessage(content=message.text))
            message_history.append(AIMessage(content=response.content))
            
            bot.reply_to(message, response.content)
        except Exception as e:
            error_message = f"Произошла ошибка: {e}"
            bot.reply_to(message, error_message)
            ic(error_message)
            traceback.print_exc()

# Start bot polling
if __name__ == "__main__":
    ic("Starting bot polling")
    bot.polling(none_stop=True)


ic| 'Starting bot polling'
ic| f"Received message: {message.text}": 'Received message: ты кто?'
ic| f"Generated response: {response.content}": ('Generated response: Я - виртуальный ассистент, специализирующийся на '
                                                'консультировании по вопросам инвестиций, мерам поддержки и инвестиционным '
                                                'проектам правительства Москвы. Могу помочь вам с информацией, касающейся '
                                                'инвестиционной деятельности в Москве. Если у вас есть какие-либо вопросы, не '
                                                'стесняйтесь задавать.')
